In [60]:
from mlflow.client import MlflowClient
import os

In [59]:
TRACKING_URI = "sqlite:///mlflow.db"
EXPERIMENT_NAME='nyc-taxi-experiment'

In [62]:
os.environ['MLFLOW_TRACKING_URI']=TRACKING_URI

In [63]:
client = MlflowClient(tracking_uri=TRACKING_URI)

In [64]:
experiments = client.search_experiments(filter_string=f"name='{EXPERIMENT_NAME}'")
print(experiments)

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025/mlruns/1', creation_time=1745700092379, experiment_id='1', last_update_time=1745700092379, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]


In [67]:
try:
    client.create_experiment(name="created-from-client")
except Exception as e:
    print(f"Experiment already exists: {e}")

Experiment already exists: Experiment(name=created-from-client) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('created-from-client', None, 'active', 1745795209903, 1745795209903)]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


In [56]:
runs= client.search_runs(
    experiment_ids=experiments[0].experiment_id,
    filter_string="metrics.rmse < 6.5",
    run_view_type=1,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"Run ID: {run.info.run_id}")
    print(f"RMSE: {run.data.metrics['rmse']}")
    print("-" * 20)

Run ID: 7999df9ae5694409907baddb7f719de1
RMSE: 5.883470547268358
--------------------
Run ID: 4b6124427c774aecb071164de764d6b3
RMSE: 5.960309272024859
--------------------
Run ID: 58e9cc0172cb49c7ab82a306afc6daac
RMSE: 6.316187412106233
--------------------


# Register Model using Client

In [37]:
model_name = "nyc-model"
model_uri = f"runs:/7999df9ae5694409907baddb7f719de1/model"


model_version = client.create_model_version(
    name=model_name,
    source=model_uri,
    run_id=run.info.run_id
)


print(f"Model version: {model_version.version}")

Model version: 2


In [66]:
import mlflow

mlflow.register_model(
    model_uri=model_uri,
    name="nyc-model",
)

Registered model 'nyc-model' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-model'.


<ModelVersion: aliases=[], creation_timestamp=1745795067738, current_stage='None', description=None, last_updated_timestamp=1745795067738, name='nyc-model', run_id='69d054aa1bf84a66a7dec8857042e52e', run_link=None, source='/workspaces/mlops-zoomcamp-2025/mlruns/1/69d054aa1bf84a66a7dec8857042e52e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>